In [1]:
import numpy as np
import math
import time
import torch
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from Modules.training_functions import *
from Modules.pennylane_functions import *

# if gpu available, set device to gpu
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Using the GPU")
else:
    device = torch.device("cpu")
    print("WARNING: Could not find GPU, using the CPU")
ld_dim=4
T=25
# load dataset
mnist_images0 = np.load(f'Data/dataset_ld_{ld_dim}_0.npy')
mnist_images1 = np.load(f'Data/dataset_ld_{ld_dim}_1.npy')

mnist_images =np.concatenate((mnist_images0, mnist_images1), axis = 0)
print(np.shape(mnist_images))

np.random.shuffle(mnist_images)
mnist_images = torch.tensor(mnist_images).to(device)

# make dataloader
data_loader = torch.utils.data.DataLoader(mnist_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
qc_array=np.array([0,2])
min_array=np.array([0.1,0.05,0.01,0.08])
layer_array=np.array([5,10,20,50])
print(NUM_QUBITS)
print(T)
for layer_indx in range(len(layer_array)):
    n_layer=layer_array[layer_indx]
    for q_indx in range(len(qc_array)):
        qc=qc_array[q_indx]
        for min_indx in range(len(min_array)):
            min_b=min_array[min_indx]

            betas      = np.insert(np.linspace(10e-8,min_b, T), 0, 0)
            print(np.shape(betas))
            alphas     = 1 - betas
            alphas_bar = np.cumprod(alphas)
            pi         = math.pi
            betas      = torch.tensor(betas).float().to(device)
            alphas     = torch.tensor(alphas).float().to(device)
            alphas_bar = torch.tensor(alphas_bar).float().to(device)
            theta_1    = Variable(torch.rand((n_layer*3*NUM_QUBITS+n_layer*3*(NUM_QUBITS)), device = device), requires_grad=True)
            optimizer = torch.optim.Adam([theta_1], lr = LEARNING_RATE)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = SCHEDULER_PATIENCE, gamma = SCHEDULER_GAMMA, verbose = False)
            trained_thetas_1 = []
            loss_history = []
            best_loss = 1e10

            for epoch in range(NUM_EPOCHS):
                print(epoch)

                t0 = time.time()
                num_batch=0
                tot_loss=0

                for image_batch in data_loader:

                    # extract batch of random times and betas
                    t = torch.randint(0, T, size = (BATCH_SIZE, ), device=device)
                    betas_batch = betas[t].to(device)
                    alphas_batch=alphas_bar[t].to(device)

                    # assemble input at t add noise (t+1)
                    target_batch = assemble_input(image_batch, t, alphas_bar,ld_dim ,device)
                    input_batch  = noise_step(target_batch, t+1, betas,ld_dim, device)
                    target_batch = target_batch / torch.norm(target_batch, dim = 1).view(-1, 1)
                    input_batch  = input_batch / torch.norm(input_batch, dim = 1).view(-1, 1)
                    #zero = torch.zeros(128, 256*7).to(device)

                    # concatenate the two tensors along the second dimension
                    #input_batch = torch.cat((input_batch, zero), dim=1)
                    #target_batch = torch.cat((target_batch, zero), dim=1)
                    # Feed to circuit, compute the loss and update the weights
                    num_batch+=1
                    loss = loss_fn_aq(qc,theta_1,n_layer, input_batch, target_batch)
                    tot_loss+=loss.item()
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                # append parameters and print loss
                trained_thetas_1.append(theta_1.cpu().clone().detach().numpy())

                loss_history.append(tot_loss/num_batch)
                if loss.item()< best_loss:
                    best_loss=loss.item()

                # implement learning rate scheduler
                scheduler.step()


            # print every epoch
                print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={((time.time()-t0)*(NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))/60:.2f} min{min_b} nl{n_layer}')
                #print(f'T={T} Epoch: {epoch+1}/{NUM_EPOCHS} - Loss: {loss.item():.4f} b_loss={best_loss:.4f} - T: {time.time()-t0:.2f}s/epoch ,tempo_previto={(((NUM_EPOCHS-1-epoch+NUM_EPOCHS*(len(qc_array)-q_indx-1)+NUM_EPOCHS*len(qc_array)*(len(min_array)-min_indx-1)+NUM_EPOCHS*len(qc_array)*len(min_array)*(len(layer_array)-layer_indx-1)))):.2f} min{min_b} nl{n_layer}')
                
            np.save(f'thetas_T{T}_nl{n_layer}_min{min_b}_qc{qc}_{Q_ANCILLA}_ld{ld_dim}.npy',trained_thetas_1)
            np.save(f'loss__T{T}_nl{n_layer}_min{min_b}_qc{qc}_ancilla{Q_ANCILLA}_ld{ld_dim}.npy',loss_history)

(14780, 4)
2
25
(26,)
0
T=25 Epoch: 1/30 - Loss: 0.0147 b_loss=0.0147 - T: 1.96s/epoch ,tempo_previto=31.36 min0.1 nl5
1
T=25 Epoch: 2/30 - Loss: 0.0111 b_loss=0.0111 - T: 1.88s/epoch ,tempo_previto=29.95 min0.1 nl5
2
T=25 Epoch: 3/30 - Loss: 0.0114 b_loss=0.0111 - T: 1.88s/epoch ,tempo_previto=29.97 min0.1 nl5
3
T=25 Epoch: 4/30 - Loss: 0.0112 b_loss=0.0111 - T: 1.89s/epoch ,tempo_previto=30.11 min0.1 nl5
4
T=25 Epoch: 5/30 - Loss: 0.0130 b_loss=0.0111 - T: 1.88s/epoch ,tempo_previto=29.86 min0.1 nl5
5
T=25 Epoch: 6/30 - Loss: 0.0117 b_loss=0.0111 - T: 1.88s/epoch ,tempo_previto=29.92 min0.1 nl5
6
T=25 Epoch: 7/30 - Loss: 0.0110 b_loss=0.0110 - T: 1.87s/epoch ,tempo_previto=29.75 min0.1 nl5
7
T=25 Epoch: 8/30 - Loss: 0.0119 b_loss=0.0110 - T: 1.95s/epoch ,tempo_previto=31.00 min0.1 nl5
8
T=25 Epoch: 9/30 - Loss: 0.0139 b_loss=0.0110 - T: 1.85s/epoch ,tempo_previto=29.34 min0.1 nl5
9
T=25 Epoch: 10/30 - Loss: 0.0133 b_loss=0.0110 - T: 1.86s/epoch ,tempo_previto=29.47 min0.1 nl5
10
T=25

T=25 Epoch: 23/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.94s/epoch ,tempo_previto=26.41 min0.01 nl5
23
T=25 Epoch: 24/30 - Loss: 0.0002 b_loss=0.0002 - T: 2.12s/epoch ,tempo_previto=28.80 min0.01 nl5
24
T=25 Epoch: 25/30 - Loss: 0.0002 b_loss=0.0002 - T: 2.15s/epoch ,tempo_previto=29.14 min0.01 nl5
25
T=25 Epoch: 26/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.95s/epoch ,tempo_previto=26.49 min0.01 nl5
26
T=25 Epoch: 27/30 - Loss: 0.0002 b_loss=0.0002 - T: 2.00s/epoch ,tempo_previto=27.06 min0.01 nl5
27
T=25 Epoch: 28/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.92s/epoch ,tempo_previto=26.05 min0.01 nl5
28
T=25 Epoch: 29/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.94s/epoch ,tempo_previto=26.19 min0.01 nl5
29
T=25 Epoch: 30/30 - Loss: 0.0002 b_loss=0.0002 - T: 1.87s/epoch ,tempo_previto=25.23 min0.01 nl5
(26,)
0
T=25 Epoch: 1/30 - Loss: 0.0946 b_loss=0.0946 - T: 1.92s/epoch ,tempo_previto=24.95 min0.08 nl5
1
T=25 Epoch: 2/30 - Loss: 0.0096 b_loss=0.0096 - T: 1.93s/epoch ,tempo_previto=25.05 min0.08 nl5
2

T=25 Epoch: 16/30 - Loss: 0.1743 b_loss=0.1486 - T: 1.89s/epoch ,tempo_previto=26.90 min0.05 nl5
16
T=25 Epoch: 17/30 - Loss: 0.1784 b_loss=0.1486 - T: 1.91s/epoch ,tempo_previto=27.14 min0.05 nl5
17
T=25 Epoch: 18/30 - Loss: 0.1719 b_loss=0.1486 - T: 1.90s/epoch ,tempo_previto=27.05 min0.05 nl5
18
T=25 Epoch: 19/30 - Loss: 0.1447 b_loss=0.1447 - T: 1.99s/epoch ,tempo_previto=28.29 min0.05 nl5
19
T=25 Epoch: 20/30 - Loss: 0.1641 b_loss=0.1447 - T: 1.87s/epoch ,tempo_previto=26.49 min0.05 nl5
20
T=25 Epoch: 21/30 - Loss: 0.1576 b_loss=0.1447 - T: 1.88s/epoch ,tempo_previto=26.67 min0.05 nl5
21
T=25 Epoch: 22/30 - Loss: 0.1446 b_loss=0.1446 - T: 1.91s/epoch ,tempo_previto=26.95 min0.05 nl5
22
T=25 Epoch: 23/30 - Loss: 0.1730 b_loss=0.1446 - T: 1.91s/epoch ,tempo_previto=26.95 min0.05 nl5
23
T=25 Epoch: 24/30 - Loss: 0.1752 b_loss=0.1446 - T: 1.91s/epoch ,tempo_previto=26.94 min0.05 nl5
24
T=25 Epoch: 25/30 - Loss: 0.1511 b_loss=0.1446 - T: 1.88s/epoch ,tempo_previto=26.42 min0.05 nl5
25


T=25 Epoch: 9/30 - Loss: 0.0103 b_loss=0.0103 - T: 3.27s/epoch ,tempo_previto=38.74 min0.1 nl10
9
T=25 Epoch: 10/30 - Loss: 0.0137 b_loss=0.0103 - T: 3.24s/epoch ,tempo_previto=38.33 min0.1 nl10
10
T=25 Epoch: 11/30 - Loss: 0.0126 b_loss=0.0103 - T: 3.26s/epoch ,tempo_previto=38.48 min0.1 nl10
11
T=25 Epoch: 12/30 - Loss: 0.0132 b_loss=0.0103 - T: 3.26s/epoch ,tempo_previto=38.51 min0.1 nl10
12
T=25 Epoch: 13/30 - Loss: 0.0120 b_loss=0.0103 - T: 3.31s/epoch ,tempo_previto=38.97 min0.1 nl10
13
T=25 Epoch: 14/30 - Loss: 0.0120 b_loss=0.0103 - T: 3.67s/epoch ,tempo_previto=43.16 min0.1 nl10
14
T=25 Epoch: 15/30 - Loss: 0.0107 b_loss=0.0103 - T: 3.24s/epoch ,tempo_previto=38.06 min0.1 nl10
15
T=25 Epoch: 16/30 - Loss: 0.0126 b_loss=0.0103 - T: 3.26s/epoch ,tempo_previto=38.27 min0.1 nl10
16
T=25 Epoch: 17/30 - Loss: 0.0113 b_loss=0.0103 - T: 3.34s/epoch ,tempo_previto=39.11 min0.1 nl10
17
T=25 Epoch: 18/30 - Loss: 0.0122 b_loss=0.0103 - T: 3.34s/epoch ,tempo_previto=39.07 min0.1 nl10
18
T=

T=25 Epoch: 1/30 - Loss: 0.0135 b_loss=0.0135 - T: 3.26s/epoch ,tempo_previto=29.32 min0.08 nl10
1
T=25 Epoch: 2/30 - Loss: 0.0088 b_loss=0.0088 - T: 3.30s/epoch ,tempo_previto=29.63 min0.08 nl10
2
T=25 Epoch: 3/30 - Loss: 0.0090 b_loss=0.0088 - T: 3.69s/epoch ,tempo_previto=33.02 min0.08 nl10
3
T=25 Epoch: 4/30 - Loss: 0.0094 b_loss=0.0088 - T: 3.34s/epoch ,tempo_previto=29.87 min0.08 nl10
4
T=25 Epoch: 5/30 - Loss: 0.0095 b_loss=0.0088 - T: 3.36s/epoch ,tempo_previto=29.94 min0.08 nl10
5
T=25 Epoch: 6/30 - Loss: 0.0088 b_loss=0.0088 - T: 3.37s/epoch ,tempo_previto=29.98 min0.08 nl10
6
T=25 Epoch: 7/30 - Loss: 0.0085 b_loss=0.0085 - T: 3.45s/epoch ,tempo_previto=30.65 min0.08 nl10
7
T=25 Epoch: 8/30 - Loss: 0.0089 b_loss=0.0085 - T: 3.30s/epoch ,tempo_previto=29.22 min0.08 nl10
8
T=25 Epoch: 9/30 - Loss: 0.0088 b_loss=0.0085 - T: 3.32s/epoch ,tempo_previto=29.36 min0.08 nl10
9
T=25 Epoch: 10/30 - Loss: 0.0107 b_loss=0.0085 - T: 3.72s/epoch ,tempo_previto=32.82 min0.08 nl10
10
T=25 Epo

...

T=25 Epoch: 7/30 - Loss: 0.0108 b_loss=0.0080 - T: 6.01s/epoch ,tempo_previto=29.37 min0.08 nl20
7
T=25 Epoch: 8/30 - Loss: 0.0100 b_loss=0.0080 - T: 5.97s/epoch ,tempo_previto=29.06 min0.08 nl20
8
T=25 Epoch: 9/30 - Loss: 0.0086 b_loss=0.0080 - T: 6.03s/epoch ,tempo_previto=29.24 min0.08 nl20
9
T=25 Epoch: 10/30 - Loss: 0.0082 b_loss=0.0080 - T: 5.96s/epoch ,tempo_previto=28.81 min0.08 nl20
10
T=25 Epoch: 11/30 - Loss: 0.0096 b_loss=0.0080 - T: 6.55s/epoch ,tempo_previto=31.53 min0.08 nl20
11
T=25 Epoch: 12/30 - Loss: 0.0100 b_loss=0.0080 - T: 5.99s/epoch ,tempo_previto=28.74 min0.08 nl20
12
T=25 Epoch: 13/30 - Loss: 0.0079 b_loss=0.0079 - T: 5.99s/epoch ,tempo_previto=28.64 min0.08 nl20
13
T=25 Epoch: 14/30 - Loss: 0.0097 b_loss=0.0079 - T: 5.99s/epoch ,tempo_previto=28.57 min0.08 nl20
14
T=25 Epoch: 15/30 - Loss: 0.0102 b_loss=0.0079 - T: 5.99s/epoch ,tempo_previto=28.47 min0.08 nl20
15
T=25 Epoch: 16/30 - Loss: 0.0093 b_loss=0.0079 - T: 6.52s/epoch ,tempo_previto=30.87 min0.08 nl20

T=25 Epoch: 29/30 - Loss: 0.1690 b_loss=0.1456 - T: 6.01s/epoch ,tempo_previto=36.17 min0.05 nl20
29
T=25 Epoch: 30/30 - Loss: 0.1629 b_loss=0.1456 - T: 6.54s/epoch ,tempo_previto=39.26 min0.05 nl20
(26,)
0
T=25 Epoch: 1/30 - Loss: 0.0652 b_loss=0.0652 - T: 6.03s/epoch ,tempo_previto=33.09 min0.01 nl20
1
T=25 Epoch: 2/30 - Loss: 0.0697 b_loss=0.0652 - T: 6.03s/epoch ,tempo_previto=32.99 min0.01 nl20
2
T=25 Epoch: 3/30 - Loss: 0.0624 b_loss=0.0624 - T: 5.98s/epoch ,tempo_previto=32.59 min0.01 nl20
3
T=25 Epoch: 4/30 - Loss: 0.0650 b_loss=0.0624 - T: 6.01s/epoch ,tempo_previto=32.67 min0.01 nl20
4
T=25 Epoch: 5/30 - Loss: 0.0651 b_loss=0.0624 - T: 6.53s/epoch ,tempo_previto=35.35 min0.01 nl20
5
T=25 Epoch: 6/30 - Loss: 0.0682 b_loss=0.0624 - T: 6.01s/epoch ,tempo_previto=32.44 min0.01 nl20
6
T=25 Epoch: 7/30 - Loss: 0.0674 b_loss=0.0624 - T: 6.11s/epoch ,tempo_previto=32.87 min0.01 nl20
7
T=25 Epoch: 8/30 - Loss: 0.0637 b_loss=0.0624 - T: 6.07s/epoch ,tempo_previto=32.58 min0.01 nl20
8
T

T=25 Epoch: 21/30 - Loss: 0.0134 b_loss=0.0097 - T: 14.55s/epoch ,tempo_previto=53.10 min0.1 nl50
21
T=25 Epoch: 22/30 - Loss: 0.0132 b_loss=0.0097 - T: 14.58s/epoch ,tempo_previto=52.99 min0.1 nl50
22
T=25 Epoch: 23/30 - Loss: 0.0119 b_loss=0.0097 - T: 14.58s/epoch ,tempo_previto=52.72 min0.1 nl50
23
T=25 Epoch: 24/30 - Loss: 0.0116 b_loss=0.0097 - T: 14.40s/epoch ,tempo_previto=51.82 min0.1 nl50
24
T=25 Epoch: 25/30 - Loss: 0.0112 b_loss=0.0097 - T: 14.60s/epoch ,tempo_previto=52.31 min0.1 nl50
25
T=25 Epoch: 26/30 - Loss: 0.0105 b_loss=0.0097 - T: 14.58s/epoch ,tempo_previto=52.01 min0.1 nl50
26
T=25 Epoch: 27/30 - Loss: 0.0097 b_loss=0.0097 - T: 14.59s/epoch ,tempo_previto=51.79 min0.1 nl50
27
T=25 Epoch: 28/30 - Loss: 0.0118 b_loss=0.0097 - T: 14.58s/epoch ,tempo_previto=51.52 min0.1 nl50
28
T=25 Epoch: 29/30 - Loss: 0.0140 b_loss=0.0097 - T: 14.58s/epoch ,tempo_previto=51.26 min0.1 nl50
29
T=25 Epoch: 30/30 - Loss: 0.0117 b_loss=0.0097 - T: 14.55s/epoch ,tempo_previto=50.91 min0.

T=25 Epoch: 12/30 - Loss: 0.0093 b_loss=0.0085 - T: 14.57s/epoch ,tempo_previto=11.66 min0.08 nl50
12
T=25 Epoch: 13/30 - Loss: 0.0100 b_loss=0.0085 - T: 14.48s/epoch ,tempo_previto=11.34 min0.08 nl50
13
T=25 Epoch: 14/30 - Loss: 0.0079 b_loss=0.0079 - T: 14.55s/epoch ,tempo_previto=11.15 min0.08 nl50
14
T=25 Epoch: 15/30 - Loss: 0.0090 b_loss=0.0079 - T: 14.69s/epoch ,tempo_previto=11.02 min0.08 nl50
15
T=25 Epoch: 16/30 - Loss: 0.0089 b_loss=0.0079 - T: 14.60s/epoch ,tempo_previto=10.71 min0.08 nl50
16
T=25 Epoch: 17/30 - Loss: 0.0108 b_loss=0.0079 - T: 14.67s/epoch ,tempo_previto=10.51 min0.08 nl50
17
T=25 Epoch: 18/30 - Loss: 0.0087 b_loss=0.0079 - T: 14.58s/epoch ,tempo_previto=10.21 min0.08 nl50
18
T=25 Epoch: 19/30 - Loss: 0.0099 b_loss=0.0079 - T: 14.52s/epoch ,tempo_previto=9.92 min0.08 nl50
19
T=25 Epoch: 20/30 - Loss: 0.0098 b_loss=0.0079 - T: 14.52s/epoch ,tempo_previto=9.68 min0.08 nl50
20
T=25 Epoch: 21/30 - Loss: 0.0090 b_loss=0.0079 - T: 14.55s/epoch ,tempo_previto=9.46

T=25 Epoch: 3/30 - Loss: 0.0731 b_loss=0.0646 - T: 14.57s/epoch ,tempo_previto=21.13 min0.01 nl50
3
T=25 Epoch: 4/30 - Loss: 0.0602 b_loss=0.0602 - T: 14.53s/epoch ,tempo_previto=20.83 min0.01 nl50
4
T=25 Epoch: 5/30 - Loss: 0.0680 b_loss=0.0602 - T: 14.63s/epoch ,tempo_previto=20.73 min0.01 nl50
5
T=25 Epoch: 6/30 - Loss: 0.0635 b_loss=0.0602 - T: 14.39s/epoch ,tempo_previto=20.14 min0.01 nl50
6
T=25 Epoch: 7/30 - Loss: 0.0601 b_loss=0.0601 - T: 14.59s/epoch ,tempo_previto=20.18 min0.01 nl50
7
T=25 Epoch: 8/30 - Loss: 0.0642 b_loss=0.0601 - T: 14.62s/epoch ,tempo_previto=19.98 min0.01 nl50
8
T=25 Epoch: 9/30 - Loss: 0.0644 b_loss=0.0601 - T: 14.48s/epoch ,tempo_previto=19.55 min0.01 nl50
9
T=25 Epoch: 10/30 - Loss: 0.0643 b_loss=0.0601 - T: 14.57s/epoch ,tempo_previto=19.43 min0.01 nl50
10
T=25 Epoch: 11/30 - Loss: 0.0774 b_loss=0.0601 - T: 14.58s/epoch ,tempo_previto=19.20 min0.01 nl50
11
T=25 Epoch: 12/30 - Loss: 0.0610 b_loss=0.0601 - T: 14.55s/epoch ,tempo_previto=18.91 min0.01 nl